This notebook aims estimating the structure factor and the pair correlation function of the Pushed Poisson point process

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 12
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=12)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# to delete this cell
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
from GPPY.spatial_windows import BallWindow, BoxWindow
from GPPY.point_processes import HomogeneousPoissonPointProcess
from GPPY.gravity_point_process import GravityPointProcess
import numpy as np


In [ ]:
from GPPY.point_pattern import PointPattern
from GPPY.spatial_windows import BoxWindow, BallWindow
import structure_factor
import os
import sys
sys.path.insert(0, os.path.abspath('../../../structure-factor/src/'))

def from_gppy_to_structure_factor_object(point_pattern):
    if isinstance(point_pattern.window, BallWindow):
        window = structure_factor.spatial_windows.BallWindow(center=point_pattern.window.center,
                    radius=point_pattern.window.radius)
    else:
        print( type(point_pattern.window))
        window = structure_factor.spatial_windows.BoxWindow(bounds=point_pattern.window.bounds)
    sf_object= structure_factor.point_pattern.PointPattern(
        points=point_pattern.points, 
        window=window,
        intensity=point_pattern.intensity)
    return sf_object.restrict_to_window(window=window)

# 1- d=2

In [ ]:
d = 2
r=100
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 1
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=1) 
print("Nb points=", poisson_pp.points.shape[0])
gppy = GravityPointProcess(poisson_pp)
push_pp = gppy.pushed_point_pattern()
epsilon = gppy.epsilon_critical


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 4))
simu_window.plot(axis=ax[0], color="grey")
poisson_pp.plot(axis=ax[0], s=0.1)
simu_window.plot(axis=ax[1], color="grey")
push_pp.plot(axis=ax[1], s=0.1)
#plt.savefig("poisson_and_push_and_force.pdf")
plt.show()

In [ ]:
push_pp_sf = from_gppy_to_structure_factor_object(push_pp)

## 1.1 Structure Factor

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_isotropic = StructureFactor(push_pp_sf)


In [ ]:
# Bartlett's isotropic estimator on allowed wavenumbers
# Running time = 156 s
import time

start_time = time.time()
k_norm_allowed, s_bi_k_norm_allowed = sf_isotropic.bartlett_isotropic_estimator(nb_values=60)
print("--- %s seconds ---" % (time.time() - start_time))

#Plot
fig, axis = plt.subplots(figsize=(7,6))
sf_isotropic.plot_isotropic_estimator(k_norm_allowed, s_bi_k_norm_allowed, 
                                         axis=axis,
                                         label=r"$\widehat{S}$")

## 1.2 Pair correlation function

In [ ]:
# pcf.fv
import structure_factor.pair_correlation_function as pcf

pcf_fv = pcf.estimate(push_pp_sf, # PointPattern 
                      method="fv", # Estimation method (could be "ppp" or "fv")
                      Kest=dict(rmax=5), # Maximal estimation radius 
                      fv=dict(method="b", spar=0.2) # Correction method and sparsity
                             )



In [ ]:
epsilon

In [ ]:
# Plot
fig, axis = plt.subplots(figsize=(5,4))
axis.plot(pcf_fv["r"], pcf_fv["pcf"], "b.", label="Push")
axis.plot(pcf_fv["r"], pcf_fv["pcf"], "b")
axis.plot(pcf_fv["r"], np.ones_like(pcf_fv["r"]), "k", label="Poisson")
axis.vlines(epsilon, ymin=0, ymax=1.2, 
             colors="red", linestyles='dashed', label=r"$\varepsilon_0$")
axis.legend()
plt.tight_layout()
plt.show()

# 2- d=3

In [ ]:
d = 3
r=50
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 1
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=1) 
print("Nb points=", poisson_pp.points.shape[0])
gppy = GravityPointProcess(poisson_pp)
push_pp = gppy.pushed_point_pattern(core_number=5)


In [ ]:
push_pp_sf_3d = from_gppy_to_structure_factor_object(push_pp)
epsilon_3d = gppy.epsilon_critical

## 2.1- Structure factor

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_isotropic_3d = StructureFactor(push_pp_sf_3d)


In [ ]:
# Bartlett's isotropic estimator on allowed wavenumbers
# Running time = 156 s
import time

start_time = time.time()
k_norm_allowed_3d, s_bi_k_norm_allowed_3d = sf_isotropic_3d.bartlett_isotropic_estimator(nb_values=60)
print("--- %s seconds ---" % (time.time() - start_time))

#Plot
fig, axis = plt.subplots(figsize=(7,6))
sf_isotropic.plot_isotropic_estimator(k_norm_allowed_3d, 
                                      s_bi_k_norm_allowed_3d, 
                                         axis=axis,
                                         label=r"$\widehat{S}$")

## 2.2- Pair correlation function

In [ ]:
# pcf.fv
import structure_factor.pair_correlation_function as pcf

pcf_fv_3d = pcf.estimate(push_pp_sf_3d, # PointPattern 
                      method="fv", # Estimation method (could be "ppp" or "fv")
                      Kest=dict(rmax=5), # Maximal estimation radius 
                      fv=dict(method="b", spar=0.2) # Correction method and sparsity
                             )

# Plot
fig, axis = plt.subplots(figsize=(7,4))
pcf.plot(pcf_fv_3d,  
         axis=axis,
         figsize=(7,6), 
         color=['grey'], 
         style=["."]
        )
axis.vlines(epsilon_3d, ymin=min(pcf_fv), ymax=max(pcf_fv), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
plt.tight_layout()
plt.show()